In [ ]:
import os
import torch, torchvision
import mmseg
import mmcv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mmcv import Config
from mmcv.runner import load_checkpoint
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
from mmseg.apis import set_random_seed
from mmseg.utils import get_device
from mmseg.datasets import build_dataset, build_dataloader
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor, single_gpu_test
from torchsummary import summary
from mmseg.core.evaluation import get_palette
from mmseg.apis import single_gpu_test
from mmseg.datasets.builder import build_dataloader
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel

import os.path as osp
import numpy as np
import cv2

import pandas as pd

import numpy as np
from PIL import Image

In [ ]:
# Load trained model
config = "configs/2023-03-20-09-19-04_Camvid11Dataset_DepthwiseSeparableFCNHead_ResNet.py"
checkpoint = "work_dirs/experimentation/2023-03-20-09-19-04_Camvid11Dataset_DepthwiseSeparableFCNHead_ResNet/best_mIoU_epoch_961.pth"
cfg = Config.fromfile(config)
datasets = [build_dataset(cfg.data.val)]

palette = datasets[0].PALETTE
classes = datasets[0].CLASSES

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint)
checkpoint['meta']['CLASSES'] = classes
checkpoint['meta']['PALETTE'] = palette

datasets = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        datasets,
        samples_per_gpu=1,
        workers_per_gpu=1)
model.CLASSES = datasets.CLASSES
model = MMDataParallel(model, device_ids=[0])

single_gpu_test(model, data_loader, out_dir="./results/"+cfg.filename)


In [ ]:
# Save inference results as images.
save_root = "inference/camvid11/resnet32-s2-fast-sccn-decoder/"
data_root = "data/CamVid11/val/"
for file in mmcv.scandir(data_root, suffix=".png"):
    img_dir = osp.join(data_root, file)
    result = inference_segmentor(model, img_dir)
    arr = np.asarray(result[0])
    img = Image.fromarray(arr.astype(np.uint8))
    
    img.save(save_root + str(file))

In [ ]:
# Print inference results
for data in data_loader:
    with torch.no_grad():
        result = inference_segmentor(model, datasets)
    print(result)
